# Static Molecular Response Properties

The response of a molecular system to an external electric or magnetic field is the essence of the fields of linear and non-linear optical spectroscopy. In this project, we explore the response of a molecular system to *static* external electric fields, and we will derive and implement expressions for molecular properties such as the electric dipole moment, polarizability, and hyperpolarizability. Consider the following expression where we have expanded the energy of a molecular system as a Taylor series in applied static external electric field ($\epsilon$) as

$$\begin{align}
E = E_0 - \mu_\alpha \epsilon_\alpha- \frac{1}{2} \alpha_{\alpha\beta} \epsilon_\alpha \epsilon_\beta - \frac{1}{6} \beta_{\alpha\beta\gamma} \epsilon_\alpha \epsilon_\beta\epsilon_\gamma  + ...
\end{align}$$

Here, $E_0$ is the unperturbed energy of the system, $\mu_\alpha$ represents cartesian components of the electric dipole moment, $\alpha_{\alpha\beta}$, $\xi_{\alpha\beta}$ is an element of the the electric dipole polarizability tensor, and $\beta_{\alpha\beta\gamma}$ is an element of the first hyperpolarizability tensor. Note that here and throughout this tutorial, repeated labels imply summation. Note also that this expression could be generalized to account for static external magnetic fields as well, in which case terms involving the magnetizability, NMR shielding tensor, etc. would appear. From this energy expression, we can see that the static response properties can be evaluated as derivatives of the energy with respect to the external field, *i.e.*,

$$\begin{align}
\mu_\alpha &= -\left .\frac{\partial E}{\partial \epsilon_\alpha}\right |_{\epsilon = 0}\\
\alpha_{\alpha\beta} &= -\left .\frac{\partial^2 E}{\partial \epsilon_\alpha\partial \epsilon_\beta}\right |_{\epsilon = 0}\\
\beta_{\alpha\beta\gamma} &= -\left .\frac{\partial^3 E}{\partial \epsilon_\alpha \partial \epsilon_\beta \partial \epsilon_\gamma}\right |_{\epsilon = 0}
\end{align}$$

Below, we will derive and implement Python code to evaluate these molecular static response properties for a molecular system described at the [Hartree-Fock](https://deprincelab.github.io/tutorials/jupyter_notebooks/hartree_fock/hartree_fock.html) level of theory.  All of the required molecular integrals will be obtained from the Psi4 package, and we will generate the required equations and Python expressions with the help of the [p$^\dagger$q package](https://github.com/edeprince3/pdaggerq).

## The Perturbed Hartree-Fock Wave Function

At the Hartree-Fock level of theory, the $N$-electron wave function is approximated as an antisymmetrized product of $N$ one-electron functions called molecular orbitals (MOs), which is called a Slater determinant that we will label as $|\Phi\rangle$. When considering the response of this wave function to an external perturbation, it is convenient to define a perturbed wave function, which can be achieved via the introduction of orbital rotation parametes, $\kappa_{ia}$, as 

$$\begin{align}
|\Phi({\kappa})\rangle = \text{exp}\left (\kappa_{ia} ( \hat{a}^\dagger_a \hat{a}_i - \hat{a}^\dagger_i \hat{a}_a ) \right ) | \Phi\rangle
\end{align}$$

The Hamiltonian corresponding to the system interacting with an external electric field is

$$\begin{align}
\hat{H} = \hat{H}_0 - \hat{\mu}_\alpha \epsilon_\alpha
\end{align}$$

where $\hat{H}_0$ represents the unperturbed Hamiltonian, and $\hat{\mu}_\alpha$ is a component of the electric dipole operator. To obtain expressions of the static response properties, we proceed by taking derivatives of the energy expression involving this Hamiltonian and the perturbed Hartree-Fock wave function, 

$$\begin{align}
E(\kappa, \epsilon) = \langle \Phi(\kappa)|\hat{H}_0 -  \hat{\mu}_\alpha \epsilon_\alpha | \Phi(\kappa)\rangle
\end{align}$$

Note that the energy depends explicitly (through the Hamiltonian) and implicitly (through the wave function parameters) on the field, and we should account for this fact via the chain rule when we differentiate, *i.e.*,

$$\begin{align}
\frac{d}{d\epsilon_\alpha} = \frac{\partial }{\partial \epsilon_\alpha} + \frac{\partial \kappa_{ia}}{\partial \epsilon_\alpha}\frac{\partial }{\partial \kappa_{ia}}
\end{align}$$

At this point, it is also useful to go ahead and consider the derivative of the perturbed wave function with respect to the external perturbation. 

$$\begin{align}
\left . \frac{d}{d \epsilon_\alpha} |\Phi({\kappa})\rangle\right |_{\epsilon=0} &= \left .\frac{\partial}{\partial \epsilon_a}|\Phi({\kappa})\rangle\right |_{\epsilon=0} +  \frac{\partial |\Phi({\kappa})\rangle }{\partial \kappa_{ia}}  \left . \frac{\partial \kappa_{ia}}{\partial \epsilon_\alpha} \right |_{\epsilon=0}\\
&= \left . ( \hat{a}^\dagger_a \hat{a}_i - \hat{a}^\dagger_i \hat{a}_a )|\Phi({\kappa_0})\rangle\frac{\partial \kappa_{ia}}{\partial \epsilon_\alpha} \right |_{\epsilon=0}
\end{align}$$
and
$$\begin{align}
\left .  \frac{d}{d \epsilon_\alpha} \langle \Phi({\kappa})| \right |_{\epsilon=0}&= \left . \left ( \frac{d}{d \epsilon_\alpha} |\Phi({\kappa})\rangle \right )^\dagger \right |_{\epsilon=0}\\
&= -\langle \Phi(\kappa_0)| ( \hat{a}^\dagger_a \hat{a}_i - \hat{a}^\dagger_i \hat{a}_a )\left . \frac{\partial \kappa_{ia}}{\partial \epsilon_\alpha}\right |_{\epsilon=0}
\end{align}$$

Here, the partial derivative of the Hartree-Fock wave function with respect to the field parameter vanishes because there is no explicit field dependence in this term, and the notation $\kappa_0$ refers to the orbital rotation parameters determined at zero field, which are the optimal Hartree-Fock parameters. 


## Expressions for the Hartree-Fock Response Properties

Now, the dipole moment is given by

$$\begin{align}
\mu_\alpha &= -\left .\frac{d E(\kappa, \epsilon)}{d \epsilon_\alpha}\right |_{\epsilon = 0} \\
&= -\left . \left (\frac{\partial }{\partial \epsilon_\alpha} + \frac{\partial \kappa_{ia}}{\partial \epsilon_\alpha}\frac{\partial }{\partial \kappa_{ia}} \right ) \langle \Phi(\kappa)|\hat{H}_0 - \hat{\mu}_\alpha \epsilon_\alpha | \Phi(\kappa)\rangle\right  |_{\epsilon = 0} \\
%&= -\left .\frac{\partial}{\partial \epsilon_\alpha} \langle \Phi(\kappa)|\hat{H}_0| \Phi(\kappa)\rangle\right  |_{\epsilon = 0} +\left .\frac{\partial}{\partial \epsilon_\alpha} \langle \Phi(\kappa)| \hat{\mu}_\alpha \epsilon_\alpha | \Phi(\kappa)\rangle\right  |_{\epsilon = 0} \\
&= \langle \Phi(\kappa_0)|( \hat{a}^\dagger_a \hat{a}_i - \hat{a}^\dagger_i \hat{a}_a )\hat{H}_0| \Phi(\kappa_0)\rangle\left . \frac{\partial \kappa_{ia}}{\partial \epsilon_\alpha}\right |_{\epsilon=0} - \langle \Phi(\kappa_0)|\hat{H}_0 ( \hat{a}^\dagger_a \hat{a}_i - \hat{a}^\dagger_i \hat{a}_a )|\Phi(\kappa_0)\rangle\left . \frac{\partial \kappa_{ia}}{\partial \epsilon_\alpha}\right |_{\epsilon=0} + \langle \Phi(\kappa_0)|\hat{\mu}|\Phi(\kappa_0)\rangle \\
&= -\langle \Phi(\kappa_0)|[\hat{H}_0,( \hat{a}^\dagger_a \hat{a}_i - \hat{a}^\dagger_i \hat{a}_a )]| \Phi(\kappa_0)\rangle\left . \frac{\partial \kappa_{ia}}{\partial \epsilon_\alpha}\right |_{\epsilon=0}  + \langle \Phi(\kappa_0)|\hat{\mu}_\alpha|\Phi(\kappa_0)\rangle \\
&= \langle \Phi(\kappa_0)|\hat{\mu}_\alpha|\Phi(\kappa_0)\rangle
\end{align}$$
where, in the last line, we have recognized that the commutator expression corresponds to the orbital gradient, which is zero for the optimal Hartree-Fock parameters. So, we have found that the electric dipole moment is simply given by the expectation value of the dipole operator with respect to the Hartree-Fock wave function (with the optimal field-free parameters).

Now, let us proceed to the case of the dipole polarizability, which is related to the second derivative of the energy with respect to the external field. We have

$$\begin{align}
\alpha_{\alpha\beta} &= -\left .\frac{d^2 E(\kappa, \epsilon)}{d \epsilon_\alpha d \epsilon_\beta}\right |_{\epsilon = 0} \\
&= \left .\left (\frac{\partial }{\partial \epsilon_\beta} + \frac{\partial \kappa_{ia}}{\partial \epsilon_\beta}\frac{\partial }{\partial \kappa_{ia}} \right ) \langle \Phi(\kappa)|\hat{\mu}_\alpha|\Phi(\kappa)\rangle \right |_{\epsilon=0}\\
&= \langle \Phi(\kappa_0)|[\hat{\mu}_\alpha,(\hat{a}^\dagger_a \hat{a}_i - \hat{a}^\dagger_i \hat{a}_a)]|\Phi(\kappa_0)\rangle \left .\frac{\partial \kappa_{ia}}{\partial \epsilon_\beta} \right |_{\epsilon=0} \\
&= 2 \mu_{ia}^{\alpha} \left .\frac{\partial \kappa_{ia}}{\partial \epsilon_\beta} \right |_{\epsilon=0}
\end{align}$$

where

$$\begin{align}
\mu_{ia}^{\alpha} = \langle \Phi(\kappa_0)|\hat{\mu}_\alpha \hat{a}^\dagger_a \hat{a}_i | \Phi(\kappa_0)\rangle = \langle \Phi(\kappa_0)|\hat{a}^\dagger_i \hat{a}_a\hat{\mu}_\alpha | \Phi(\kappa_0)\rangle
\end{align}$$

Oh, no! In order to determine the electric dipole polarizability, we must first determine the first-order response of the wave function parameters to the perturbation, $\left .\frac{\partial \kappa_{ia}}{\partial \epsilon_\beta} \right |_{\epsilon=0}$! We can find an equation to determine this quantity from the orbital gradient for the perturbed Hamiltonian, which should be zero at zero field, i.e.,

$$\begin{align}
0 = \left . \langle \Phi(\kappa_0)|[\hat{H}_0 - \mu_\alpha \epsilon_\alpha,( \hat{a}^\dagger_a \hat{a}_i - \hat{a}^\dagger_i \hat{a}_a )]| \Phi(\kappa_0)\rangle \right |_{\epsilon=0}
\end{align}$$

We differentiate this expression with respect to the external field to obtain

$$\begin{align}
0 &= \left . \frac{d}{\epsilon_\beta}  \langle \Phi(\kappa)|[\hat{H}_0 - \mu_\alpha \epsilon_\alpha,( \hat{a}^\dagger_a \hat{a}_i - \hat{a}^\dagger_i \hat{a}_a )]| \Phi(\kappa)\rangle \right |_{\epsilon=0} \\
&= \left . \left ( \frac{\partial}{\partial \epsilon_\beta} + \frac{\partial \kappa_{jb}}{\partial \epsilon_\beta} \frac{\partial }{\partial \kappa_{jb}}\right ) \langle \Phi(\kappa)|[\hat{H}_0 - \mu_\alpha \epsilon_\alpha,( \hat{a}^\dagger_a \hat{a}_i - \hat{a}^\dagger_i \hat{a}_a )]| \Phi(\kappa)\rangle \right |_{\epsilon=0} \\
&=  \langle \Phi(\kappa_0)|[[\hat{H}_0,( \hat{a}^\dagger_a \hat{a}_i - \hat{a}^\dagger_i \hat{a}_a )],(\hat{a}^\dagger_b \hat{a}_j - \hat{a}^\dagger_j \hat{a}_b)]| \Phi(\kappa_0)\rangle \left . \frac{\partial \kappa_{jb}}{\partial \epsilon_\beta} \right |_{\epsilon = 0} - \langle \Phi(\kappa_0)|[\mu_\alpha \epsilon_\alpha,( \hat{a}^\dagger_a \hat{a}_i - \hat{a}^\dagger_i \hat{a}_a )]| \Phi(\kappa_0)\rangle
\end{align}$$

or

$$\begin{align}
\langle \Phi(\kappa_0)|[[\hat{H}_0,( \hat{a}^\dagger_a \hat{a}_i - \hat{a}^\dagger_i \hat{a}_a )],(\hat{a}^\dagger_b \hat{a}_j - \hat{a}^\dagger_j \hat{a}_b)]| \Phi(\kappa_0)\rangle \left . \frac{\partial \kappa_{jb}}{\partial \epsilon_\beta} \right |_{\epsilon = 0}  = 2 \mu_{ia}^\alpha
\end{align}$$

So, in order to determine the first-order response of the wave function, we must solve a set of linear equations involving the orbital hessian (the double commutator expression) and the dipole integrals.  Defining

$$\begin{align}
h_{jb, ia} = \langle \Phi(\kappa_0)|[[\hat{H}_0,( \hat{a}^\dagger_a \hat{a}_i - \hat{a}^\dagger_i \hat{a}_a )],(\hat{a}^\dagger_b \hat{a}_j - \hat{a}^\dagger_j \hat{a}_b)]| \Phi(\kappa_0)\rangle
\end{align}$$

and assuming we can store and invert this tensor, we could determine the response vector as

$$\begin{align}
\left . \frac{\partial \kappa_{jb}}{\partial \epsilon_\beta} \right |_{\epsilon = 0}  = \left ( h_{jb, ia} \right )^{-1} 2 \mu_{ia}^\alpha
\end{align}$$

Here, we have expressed all quantities in a basis of spin orbitals. If we explicitly account for spin, then we have two response equations for $\alpha$- and $\beta$-spin:

$$\begin{align}
\left . \frac{\partial \kappa_{j_\sigma b_\sigma}}{\partial \epsilon_\beta} \right |_{\epsilon = 0}  =  \left ( h_{j_\sigma b_\sigma, i_\tau a_\tau} \right )^{-1} 2 \mu_{i_\tau a_\tau}^\alpha
\end{align}$$


We can use the p$^\dagger$q package to evaluate the different spin blocks of $h_{jb,ia}$ using the following code. 


In [1]:
import pdaggerq

from pdaggerq.parser import contracted_strings_to_tensor_terms

pq = pdaggerq.pq_helper('fermi')

pq.add_double_commutator( 1.0, ['f'], ['a*(a)', 'a(i)'], ['a*(b)', 'a(j)'])
pq.add_double_commutator(-1.0, ['f'], ['a*(i)', 'a(a)'], ['a*(b)', 'a(j)'])
pq.add_double_commutator(-1.0, ['f'], ['a*(a)', 'a(i)'], ['a*(j)', 'a(b)'])
pq.add_double_commutator( 1.0, ['f'], ['a*(i)', 'a(a)'], ['a*(j)', 'a(b)'])

pq.add_double_commutator( 1.0, ['v'], ['a*(a)', 'a(i)'], ['a*(b)', 'a(j)'])
pq.add_double_commutator(-1.0, ['v'], ['a*(i)', 'a(a)'], ['a*(b)', 'a(j)'])
pq.add_double_commutator(-1.0, ['v'], ['a*(a)', 'a(i)'], ['a*(j)', 'a(b)'])
pq.add_double_commutator( 1.0, ['v'], ['a*(i)', 'a(a)'], ['a*(j)', 'a(b)'])

pq.simplify()

for s1 in ['a', 'b']:
    for s2 in ['a', 'b']:

        spin_labels = {
            'i': s1,
            'a': s1,
            'j': s2,
            'b': s2,
        }
        terms = pq.strings(spin_labels = spin_labels)
        terms = contracted_strings_to_tensor_terms(terms)

        print('')
        print('%s-%s spin block' % (s1, s2))
        print('')
        for term in terms:
            print('#', term)
            print("%s" % (term.einsum_string(update_val='mat', output_variables=('j', 'b', 'i', 'a'))))
            print()



a-a spin block

# -1.00 d(a,b)*f_aa(j,i)
mat += -1.00 * einsum('ab,ji->jbia', kd_aa[va, va], f_aa[oa, oa])

#  1.00 d(i,j)*f_aa(a,b)
mat +=  1.00 * einsum('ij,ab->jbia', kd_aa[oa, oa], f_aa[va, va])

# -1.00 d(b,a)*f_aa(i,j)
mat += -1.00 * einsum('ba,ij->jbia', kd_aa[va, va], f_aa[oa, oa])

#  1.00 d(j,i)*f_aa(b,a)
mat +=  1.00 * einsum('ji,ba->jbia', kd_aa[oa, oa], f_aa[va, va])

# -1.00 <j,i||a,b>_aaaa
mat += -1.00 * einsum('jiab->jbia', g_aaaa[oa, oa, va, va])

#  1.00 <j,a||b,i>_aaaa
mat +=  1.00 * einsum('jabi->jbia', g_aaaa[oa, va, va, oa])

#  1.00 <i,b||a,j>_aaaa
mat +=  1.00 * einsum('ibaj->jbia', g_aaaa[oa, va, va, oa])

# -1.00 <a,b||j,i>_aaaa
mat += -1.00 * einsum('abji->jbia', g_aaaa[va, va, oa, oa])


a-b spin block

#  1.00 <i,j||a,b>_abab
mat +=  1.00 * einsum('ijab->jbia', g_abab[oa, ob, va, vb])

#  1.00 <a,j||i,b>_abab
mat +=  1.00 * einsum('ajib->jbia', g_abab[va, ob, oa, vb])

#  1.00 <i,b||a,j>_abab
mat +=  1.00 * einsum('ibaj->jbia', g_abab[oa, vb, va, ob])

#  

In the code snippet, 'f' and 'v' refer to the Fock operator ($\hat{f}$) and fluctuation potential ($\hat{v}$), respectively, and $\hat{H} = \hat{f} + \hat{v}$. In the output, 'd(a,b)' is a Kronecker delta function, 'f(a,b)' is an element of the Fock matrix, and '<j,a||b,i>' refers to an antisymmetrized electron repulsion integral (ERI) in physicists' notation.

At this point, we have all of the ingredients we need to compute the electric dipole polarizability. The following code snippets set up and run a Hartree-Fock calculation using Psi4 so that we can extract all of the required integrals.

In [2]:
import psi4
import numpy as np

# set molecule
mol = psi4.geometry("""
O  0.00000  0.00000   0.12716
H  0.00000  0.75808  -0.50864
H  0.00000 -0.75808  -0.50864
symmetry c1
""")   

psi4.set_options({'basis': 'sto-3g',
                  'scf_type': 'pk',
                  'e_convergence': 1e-12,
                  'd_convergence': 1e-12})

psi4.core.be_quiet()

# compute the Hartree-Fock energy and wavefunction
scf_e, wfn = psi4.energy('SCF', return_wfn=True)

Note that we asked the energy routine to return a wavefunction. This object contains all of the important information from the Hartree-Fock computation, including the number of electrons, the number of orbitals, the AO/MO transformation matrices, and more! We will need some of this information.

In [5]:
# Grab data from wavfunction

# all orbitals
Ca = wfn.Ca()
Cb = wfn.Cb()

# use Psi4's MintsHelper to generate ERIs and dipole integrals
mints = psi4.core.MintsHelper(wfn.basisset())

# build the integrals in chemists' notation
g = np.asarray(mints.mo_eri(Ca, Ca, Ca, Ca))

# antisymmetrized integrals in physicists' notation
g_aaaa = g.transpose(0, 2, 1, 3) - g.transpose(0, 2, 3, 1)
g_bbbb = g_aaaa
g_abab = g.transpose(0, 2, 1, 3)

nalpha = wfn.nalpha()
nbeta = wfn.nbeta()

oa = slice(None, nalpha)
ob = slice(None, nbeta)
va = slice(nalpha, None)
vb = slice(nbeta, None)

nmo = wfn.nmo()
kd_aa = np.eye(nmo)
kd_bb = np.eye(nmo)

# orbital energies
f_aa = np.diag(wfn.epsilon_a())
f_bb = np.diag(wfn.epsilon_b())

# dipole integrals
mu = mints.ao_dipole()

mu_x = np.asarray(mu[0])
mu_y = np.asarray(mu[1])
mu_z = np.asarray(mu[2])

# transform the dipole integrals to the MO basis
Ca = np.asarray(Ca)
Cb = np.asarray(Cb)

mua_x = Ca.T @ mu_x @ Ca
mua_y = Ca.T @ mu_y @ Ca
mua_z = Ca.T @ mu_z @ Ca

mub_x = Cb.T @ mu_x @ Cb
mub_y = Cb.T @ mu_y @ Cb
mub_z = Cb.T @ mu_z @ Cb

Now, you shoul have all of the ingredients to evaluate the dipole polarizability ... good luck!